In [ ]:
import os
from IPython.display import display, Markdown
import pandas as pd
from typing import List
from pydantic import BaseModel, ConfigDict
import instructor
from datasets import Dataset

import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from llama_index.core import Document, VectorStoreIndex, Settings, StorageContext, load_index_from_storage
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss

import deepeval
from deepeval.models import DeepEvalBaseLLM, DeepEvalBaseEmbeddingModel
from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset
from deepeval.synthesizer import Synthesizer
from deepeval import evaluate
from deepeval.evaluate import TestResult, print_test_result
from deepeval.metrics import (
    AnswerRelevancyMetric,
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric,
    FaithfulnessMetric
)
from deepeval.metrics.ragas import (
    RagasMetric,
    RAGASAnswerRelevancyMetric,
    RAGASFaithfulnessMetric, 
    RAGASContextualRecallMetric,
    RAGASContextualPrecisionMetric,
    RAGASContextualRelevancyMetric
)

In [2]:
# Environmental variable to opt out of DeepEval tracking telemetry data
os.environ["DEEPEVAL_TELEMETRY_OPT_OUT"] = "YES"

In [3]:
deepeval.telemetry_opt_out()

True

In [4]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [5]:
# set up local API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [7]:
# create document database
# using 4 State of the Union speeches, all text from whitehouse.gov briefing room speeches posted online, including a title with the date of the speech
# Example from 2024:
# https://www.whitehouse.gov/briefing-room/speeches-remarks/2024/03/07/remarks-of-president-joe-biden-state-of-the-union-address-as-prepared-for-delivery-2/
sotu = []
newfiles = ["./Speeches/titleedits/state_of_the_union_042921.txt", "./Speeches/titleedits/state_of_the_union_030122.txt", "./Speeches/titleedits/state_of_the_union_020723.txt", "./Speeches/titleedits/state_of_the_union_030724.txt"]
for i in newfiles:
    with open(i) as file:
        for line in file:
            nl = line.rstrip()
            if nl != '':
                sotu.append(nl)

In [8]:
documents = [Document(text=line) for line in sotu]

In [9]:
# Set up the faiss index
d = 768 # dimensions of the input vector of the embedding model that we're going to use; in this case, the google embedding model
faiss_index = faiss.IndexFlatL2(d)
print(faiss_index.is_trained)

True


In [10]:
# set up the embeddings
doc_embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004") # optional: task_type="RETRIEVAL_DOCUMENT"
Settings.embed_model = doc_embeddings
Settings.llm = llm

In [11]:
# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./storage"
)
# index id 'cef7ae30-ff1e-404a-bce6-85d59ca4b376' uses the speeches with a title that includes the date it was given
index = load_index_from_storage(storage_context=storage_context, index_id='cef7ae30-ff1e-404a-bce6-85d59ca4b376')

In [12]:
# set up query and chat engines
query_engine = index.as_query_engine(similarity_top_k=10)
chat_engine = index.as_chat_engine(similarity_top_k=10, chat_mode='context')

In [13]:
# DeepEval requires a json response. In practice, this has led to malformed json returned from the llm, even with as simple of a schema as this
class Response(BaseModel):
    response: str

In [22]:
# Non Open-AI requiere a custom LLM class for using DeepEval
class CustomGeminiFlash(DeepEvalBaseLLM):
    def __init__(self):
        self.model = genai.GenerativeModel(model_name="models/gemini-1.5-flash")
        model_config  = ConfigDict(protected_namespaces=())

    def load_model(self):
        return self.model

    def generate(self, prompt: str, schema: BaseModel) -> BaseModel: 
        client = self.load_model()
        instructor_client = instructor.from_gemini(
            client=client,
            mode=instructor.Mode.GEMINI_JSON,
        )
        resp = instructor_client.messages.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            response_model=schema,
        )
        return resp

    async def a_generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        return self.generate(prompt, schema)

    def get_model_name(self):
        return "Gemini 1.5 Flash"

In [23]:
# similarly, a custom embedding model class is required for non Open-AI embeddings
class CustomGeminiEmbeddingModel(DeepEvalBaseEmbeddingModel):
    def __init__(self):
        model_config  = ConfigDict(protected_namespaces=())

    def load_model(self):
        return GoogleGenerativeAIEmbeddings(
            model="models/text-embedding-004"
        )

    def embed_text(self, text: str) -> List[float]:
        embedding_model = self.load_model()
        return embedding_model.embed_query(text)

    def embed_texts(self, texts: List[str]) -> List[List[float]]:
        embedding_model = self.load_model()
        return embedding_model.embed_documents(texts)

    async def a_embed_text(self, text: str) -> List[float]:
        embedding_model = self.load_model()
        return await embedding_model.aembed_query(text)

    async def a_embed_texts(self, texts: List[str]) -> List[List[float]]:
        embedding_model = self.load_model()
        return await embedding_model.aembed_documents(texts)

    def get_model_name(self):
        "Custom Gemini Embeddings"

In [24]:
custom_geminiflash = CustomGeminiFlash()
custom_geminiembeddings = CustomGeminiEmbeddingModel()

In [ ]:
# Generate a synthetic dataset of "Goldens" (aka a dataset with 'input', 'context', 'source_file' columns -- not 'Retrieval_Context') with DeepEval
dataset = EvaluationDataset()
synthesizer = Synthesizer(model=custom_geminiflash, embedder=custom_geminiembeddings)
dataset.generate_goldens_from_docs(
    synthesizer=synthesizer,
    document_paths=['Speeches/titleedits/state_of_the_union_042921.txt', 'Speeches/titleedits/state_of_the_union_030122.txt', 
                    'Speeches/titleedits/state_of_the_union_020723.txt', 'Speeches/titleedits/state_of_the_union_030724.txt'],
    max_goldens_per_document=3,
    include_expected_output=True
)

dataset.save_as(file_type="csv", directory=".")

In [22]:
# after dataset is generated, need to generate the answer column

In [37]:
# Below code uses the resulting testset without generated answers to generate new answers

#  use if imported testset_pd from csv without answers
# this is a fix for 'contexts' column being saved as a string; needs to be a list
testset_pd = pd.read_csv("20241031_162438.csv", index_col = None)
#testset_pd['contexts'] = testset_pd['contexts'].apply(ast.literal_eval)

In [38]:
testset_pd

,input,actual_output,expected_output,context,source_file
0,How has the American Rescue Plan impacted citi...,NaN,The American Rescue Plan has delivered food an...,suffering from an autoimmune disease — wrote ...,Speeches/titleedits/state_of_the_union_042921.txt
1,If the American Rescue Plan had been enacted e...,NaN,The speech does not directly address whether t...,suffering from an autoimmune disease — wrote ...,Speeches/titleedits/state_of_the_union_042921.txt
2,Compare the economic policies advocated by the...,NaN,The president advocates for economic policies ...,keep the economy going strong by giving worke...,Speeches/titleedits/state_of_the_union_030122.txt
3,Imagine if increasing the minimum wage to $15 ...,NaN,The impact of raising the minimum wage to $15 ...,keep the economy going strong by giving worke...,Speeches/titleedits/state_of_the_union_030122.txt
4,Compare President Biden's statements about wor...,NaN,President Biden emphasizes his desire to work ...,State of the Union Address given by President ...,Speeches/titleedits/state_of_the_union_020723.txt
5,"Imagine the US facing economic instability, gl...",NaN,President Biden's address emphasizes unity and...,State of the Union Address given by President ...,Speeches/titleedits/state_of_the_union_020723.txt
6,Imagine a year where America's economy is thri...,NaN,The President's speech paints a picture of a s...,no place in America! \n\nHistory is watching....,Speeches/titleedits/state_of_the_union_030724.txt
7,How does the president's statement about overt...,NaN,The president states that his predecessor brag...,no place in America! \n\nHistory is watching....,Speeches/titleedits/state_of_the_union_030724.txt


In [39]:
testset_pd = testset_pd.rename(columns={"input": "Query", "actual_output": "Answer", "expected_output": "Expected_Output", "context": "Contexts", "source_file": "Source_File"})

In [40]:
testset_pd

,Query,Answer,Expected_Output,Contexts,Source_File
0,How has the American Rescue Plan impacted citi...,NaN,The American Rescue Plan has delivered food an...,suffering from an autoimmune disease — wrote ...,Speeches/titleedits/state_of_the_union_042921.txt
1,If the American Rescue Plan had been enacted e...,NaN,The speech does not directly address whether t...,suffering from an autoimmune disease — wrote ...,Speeches/titleedits/state_of_the_union_042921.txt
2,Compare the economic policies advocated by the...,NaN,The president advocates for economic policies ...,keep the economy going strong by giving worke...,Speeches/titleedits/state_of_the_union_030122.txt
3,Imagine if increasing the minimum wage to $15 ...,NaN,The impact of raising the minimum wage to $15 ...,keep the economy going strong by giving worke...,Speeches/titleedits/state_of_the_union_030122.txt
4,Compare President Biden's statements about wor...,NaN,President Biden emphasizes his desire to work ...,State of the Union Address given by President ...,Speeches/titleedits/state_of_the_union_020723.txt
5,"Imagine the US facing economic instability, gl...",NaN,President Biden's address emphasizes unity and...,State of the Union Address given by President ...,Speeches/titleedits/state_of_the_union_020723.txt
6,Imagine a year where America's economy is thri...,NaN,The President's speech paints a picture of a s...,no place in America! \n\nHistory is watching....,Speeches/titleedits/state_of_the_union_030724.txt
7,How does the president's statement about overt...,NaN,The president states that his predecessor brag...,no place in America! \n\nHistory is watching....,Speeches/titleedits/state_of_the_union_030724.txt


In [41]:
# generate answer column, per these two issues
# https://github.com/explodinggradients/ragas/issues/1145
# https://github.com/explodinggradients/ragas/issues/1084#issuecomment-2248219601

query_engine = index.as_query_engine(similarity_top_k=10)
answers = [query_engine.query(q) for q in testset_pd['Query']]

In [42]:
# parse out new 'answer' and 'contexts' columns
answers_r = []
context_n = []
for i in answers:
    answers_r.append(i.response)
    context_n.append([c.node.get_content() for c in i.source_nodes])

testset_pd = testset_pd.rename(columns={"Contexts":"Contexts_QueryGen"})
testset_pd['Contexts'] = context_n
testset_pd['Answer'] = answers_r

In [43]:
testset_pd

,Query,Answer,Expected_Output,Contexts_QueryGen,Source_File,Contexts
0,How has the American Rescue Plan impacted citi...,The American Rescue Plan has provided financia...,The American Rescue Plan has delivered food an...,suffering from an autoimmune disease — wrote ...,Speeches/titleedits/state_of_the_union_042921.txt,[That’s why the Rescue Plan is delivering food...
1,If the American Rescue Plan had been enacted e...,The context discusses the American Rescue Plan...,The speech does not directly address whether t...,suffering from an autoimmune disease — wrote ...,Speeches/titleedits/state_of_the_union_042921.txt,[The American Rescue Plan gave schools money t...
2,Compare the economic policies advocated by the...,The speaker advocates for policies that focus ...,The president advocates for economic policies ...,keep the economy going strong by giving worke...,Speeches/titleedits/state_of_the_union_030122.txt,[But that trickle-down theory led to weaker ec...
3,Imagine if increasing the minimum wage to $15 ...,The context suggests that raising the minimum ...,The impact of raising the minimum wage to $15 ...,keep the economy going strong by giving worke...,Speeches/titleedits/state_of_the_union_030122.txt,"[Look, our economy roared back faster than mos..."
4,Compare President Biden's statements about wor...,President Biden consistently emphasizes the im...,President Biden emphasizes his desire to work ...,State of the Union Address given by President ...,Speeches/titleedits/state_of_the_union_020723.txt,"[To my Republican friends, if we could work to..."
5,"Imagine the US facing economic instability, gl...",President Biden would likely emphasize the nee...,President Biden's address emphasizes unity and...,State of the Union Address given by President ...,Speeches/titleedits/state_of_the_union_020723.txt,"[Tonight, I come to talk about crisis and oppo..."
6,Imagine a year where America's economy is thri...,Despite a robust economy with record job growt...,The President's speech paints a picture of a s...,no place in America! \n\nHistory is watching....,Speeches/titleedits/state_of_the_union_030724.txt,"[Look, our economy roared back faster than mos..."
7,How does the president's statement about overt...,The provided text does not mention Roe v. Wade...,The president states that his predecessor brag...,no place in America! \n\nHistory is watching....,Speeches/titleedits/state_of_the_union_030724.txt,"[Meanwhile, my predecessor told the NRA he’s p..."


In [44]:
testset_pd.to_csv('datasets/synth_3.csv', index=False)